# Process images to extract spectral features

April 13, 2020

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/LBANN/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

[NbConvertApp] WARNING | pattern 'modules_image_analysis.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook outp

In [5]:
### Get imput image

##### From file #####
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'
f1='20200409_083646_exagan_with_mcr/dump_outs/model0-validation-epoch45-step7544-inp_img-output0.npy'

fname=main_dir+f1
a1=np.load(fname)
img=a1[0].reshape(128,128)
print(img.shape)

(128, 128)


In [5]:
mean,cov=[64,64],[[10.0,0.0],[0.0,10.0]]
mean,cov=[64,15],[[10.0,0.0],[0.0,0.0001]]

### Not working correctly !! 
a1=np.random.multivariate_normal(mean, cov, size=int(1e6))
### Compute histogram of samples to get single 128x128 image
H,e1,e2=np.histogram2d(a1[:,0],a1[:,1],bins=(128,128),normed=True)
# H=H*(128.0*128)/np.mean(H)
print(np.mean(H))

0.3504612364550701


In [7]:
### Plot image
plt.figure()
plt.imshow(H,origin=(0,0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
f_pixel_intensity(img,bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([1.56264685e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 5.87785480e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.61552248e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 6.81355853e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.93089878e-01, 0.00000000e+00, 0.00000000e+00,
       2.38936962e-01, 0.00000000e+00, 1.43362204e-01, 0.00000000e+00,
       1.14073152e-01, 8.94086866e-02, 5.70365759e-02, 0.00000000e+00,
       4.00797561e-02, 8.01595121e-02, 1.84983350e-02, 4.16213165e-02,
       2.31229188e-02, 3.69966701e-02, 4.31628467e-02, 2.31229188e-02,
       2.62060141e-02, 2.15813909e-02, 2.00398931e-02, 1.69568071e-02,
       1.69568326e-02, 1.07906954e-02])

In [ ]:
# plt.figure()
# plt.plot(a1[:,0],linestyle='',marker='o')
# plt.plot(a1[:,1],linestyle='',marker='*')

In [11]:
from scipy.fftpack import fft2, ifft2
from mpl_toolkits.mplot3d import Axes3D

In [1]:
### 3d plot of image 


# x,y=e1[1:],e2[1:]
# fig = plt.figure()
# ax = Axes3D(fig)
# surf = ax.plot_surface(x, y, np.abs(H), cmap='viridis')

In [22]:
### new code 
# https://stackoverflow.com/questions/48842320/what-is-the-best-way-to-calculate-radial-average-of-the-image-with-python
# https://stackoverflow.com/questions/21242011/most-efficient-way-to-calculate-radial-profile

def radial_profile(data, center):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    return radialprofile


### Test with azimuthal average and compare times

def f_get_azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    # Create a grid of points with x and y coordinates
    y, x = np.indices(image.shape)
    
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
    
#     print(center)
    # Get the radial coordinate for every grid point. Array has the shape of image
    r = np.hypot(x - center[0], y - center[1])

    ind = np.argsort(r.flat) ### Get indices that sort the "r" array in ascending order.

    r_sorted = r.flat[ind]     ### Sort the "r" array
    
    i_sorted = image.flat[ind]   ### Sort the image points according to the radial coordinate

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]

    radial_prof = tbin / nr

    return radial_prof

In [14]:
%%timeit 
a1=f_get_azimuthalAverage(img)

1.81 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit 
a2=radial_profile(img,center=(63.5,63.5))

322 µs ± 4.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
# img=np.arange(100).reshape(10,10)
# img=np.ones(100).reshape(10,10)

a1=f_get_azimuthalAverage(img)
a2=radial_profile(img,center=None)


In [64]:
data=img[:]

y, x = np.indices((data.shape))
center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
r = r.astype(np.int)

tbin = np.bincount(r.ravel(), data.ravel())
nr = np.bincount(r.ravel())
radialprofile = tbin / nr

In [21]:
a1,a2

(array([-0.85000001, -0.84380953, -0.76666667, -0.73877552, -0.59841271,
        -0.75757576, -0.70185142, -0.81686508, -0.7968254 , -0.70640014,
        -0.70305996, -0.63694924, -0.53395613, -0.5461315 , -0.63739664,
        -0.62434329, -0.7347416 , -0.7434315 , -0.84095239, -0.83761273,
        -0.78987233, -0.83722618, -0.80098246, -0.828271  , -0.84079745,
        -0.79706863, -0.82936508, -0.78569298, -0.74168442, -0.75848853,
        -0.77984589, -0.80347986, -0.79418132, -0.79057705, -0.7460928 ,
        -0.77762501, -0.7915731 , -0.82635716, -0.75443458, -0.7835024 ,
        -0.78811584, -0.78379731, -0.76277959, -0.79901277, -0.80008517,
        -0.76060246, -0.78811938, -0.82529098, -0.79369129, -0.77993711,
        -0.78664353, -0.76312844, -0.74121348, -0.73358099, -0.72692457,
        -0.74036909, -0.73858259, -0.70594484, -0.71721987, -0.74173955,
        -0.77715071, -0.77844584, -0.78007353, -0.77696743, -0.8012156 ,
        -0.79799837, -0.80301005, -0.80965672, -0.7

In [66]:
np.bincount(r.ravel(),data.ravel())
# np.bincount(data.ravel())

array([ 4.,  8., 20., 20., 28., 16.,  4.])

In [25]:
np.array_equal(a1,a2[1:-1])

True

In [24]:
plt.figure()
plt.plot(a1,color='r',marker='o',linestyle='',markersize=10)
plt.plot(a2[1:],color='b',marker='*',linestyle='')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To do : 
- Read an image
- Compute radial profile
- Compute 2D fourier transform
- Compute radial profile of Mod of fourier transform -> Power spectrum



In [12]:
105060*0.1/(2.*128)

41.0390625

## 2-point function

In [80]:
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

In [78]:
plt.figure()
plt.imshow(img)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
_=f_radial_profile(img,center=(0,0))

In [119]:
a1,a2=128,128
# a1,a2=5,5
centers=np.array([(x,y) for y in np.arange(a2) for x in np.arange(a1)])

In [121]:

r_list=[]
for center in centers:
    r1=f_radial_profile(img,center=center)
    r_list.append(np.pad(r1,(0,181-len(r1)),'constant')) ## Padding with zeros at the end

r_profs=np.array(r_list)
print(r_profs.shape)

(16384, 181)


In [122]:
### compute mean and err
mean,err=np.mean(r_profs,axis=0),np.std(r_profs,axis=0)/np.sqrt(r_profs.shape[0])


In [123]:
plt.figure()
plt.errorbar(x=np.arange(mean.shape[0]),y=mean,yerr=err,linestyle='',marker='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>